<a href="https://colab.research.google.com/github/SamNavdeep/24b1303_AIC/blob/main/AIC_question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import struct
import numpy as np
import torch
from torch import nn, optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from PIL import Image
from IPython import get_ipython
from IPython.display import display
import random
import tensorflow as tf

In [ ]:
def load_images(file_path):
    with open(file_path, 'rb') as f:
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        image_data_size = num * rows * cols
        images = np.frombuffer(f.read(), dtype=np.uint8)
        images = images.reshape(num, rows, cols)
    return images

def load_labels(file_path):
    with open(file_path, 'rb') as f:
        magic, num = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels


In [ ]:
images = load_images("/content/train-images-idx3-ubyte")
labels = load_labels("/content/train-labels-idx1-ubyte")

FileNotFoundError: [Errno 2] No such file or directory: '/content/train-images-idx3-ubyte'

In [ ]:
print(len(images))
less_images = images[:6000]
less_labels = labels[:6000]

NameError: name 'images' is not defined

In [ ]:
# transform = transforms.Compose([
#     transforms.Resize((224, 224)), #you are chaanging the size
#     transforms.Grayscale(num_output_channels=3),  # Convert 1 to 3 channels
#     transforms.ToTensor(),
#     transforms.Normalize([0.5]*3, [0.5]*3)# normalisation is kinda operation which spreads the data evenly b/n -1 and 1. 0.5 and 0.5 represent mean and std dev for r,g,b valus.
# ])

from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])
x_train = transform(Image.fromarray(images))
y_train = torch.tensor(labels)


In [ ]:
import random
random.shuffle(x_train,y_train)
n= int((0.8)*len(images))
train_images = images[:n]
train_labels = labels[:n]
val_images = images[n:]
val_labels = labels[n:]
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
train_dataset =TensorDataset(train_images,train_labels)
val_dataset = TensorDataset(val_images,val_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

TypeError: Random.shuffle() takes 2 positional arguments but 3 were given

In [ ]:
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

def evaluate(model, loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for images, batch in loader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total += batch.size(0)
            correct += (preds == y).sum().item()
    return correct / total
train_accs, val_accs = [], []
epochs=5
for epoch in range(epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    train_acc = evaluate(model, train_loader)
    val_acc = evaluate(model, val_loader)
    train_accs.append(train_acc)
    val_accs.append(val_acc)

for param in model.parameters():
    param.requires_grad = True


for epoch in range(3):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    val_acc = evaluate(model, val_loader)